In [1]:
import os
import csv
from countries_languages import country_to_language
import pandas as pd
from difflib import get_close_matches
import numpy as np
import re
from transliterate import translit
from unidecode import unidecode
import Levenshtein
import requests
import bs4
from bs4 import BeautifulSoup 
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import urllib.parse
from datetime import datetime
from functools import reduce
from urllib.parse import urlsplit

import wikipediaapi

import wikipedia

from urllib.parse import urlparse

import difflib

from datetime import datetime

import calendar

from selenium import webdriver
from selenium.webdriver.chrome.options import Options

import AssemblyHelpers
#from AssemblyHelpers import find_money_info_from_name


import requests
from bs4 import BeautifulSoup

def load_csv_dataset(file_path):
    return pd.read_csv(file_path)

countries_codes = load_csv_dataset('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/countries_and_codes.csv')


#leagues value
leagues_value = load_csv_dataset('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/Most Updated Edited Transfermarkt Dataset.csv')
#leagues value large 
leagues_value_large = load_csv_dataset('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/most_updated_transfermarkt_dataset.csv')


/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/AssemblyHelpers.py:54: DtypeWarning: Columns (3,5) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(file_path)
/var/folders/vw/z2xv57_n70l_szw_fr80j5km0000gn/T/ipykernel_33244/714062244.py:49: DtypeWarning: Columns (3,5) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(file_path)


## Trying on Group 1 

In [18]:
#FUNCTIONS


import pandas as pd
from fuzzywuzzy import process
from Levenshtein import distance
import re
import unidecode

def check_names_type(item):
    if isinstance(item, list):
        return "List of Names"
    elif isinstance(item, str):
        return "Single Name"
    else:
        return "Other"

def convert_currency(value):
    # Strip the Euro sign and whitespace + convert to float and parse k/m
    if value == "-":
        return 0  
    value = value.replace('€', '').strip()
    if value[-1].lower() == 'm':
        return float(value[:-1]) * 1000000
    elif value[-1].lower() == 'k':
        return float(value[:-1]) * 1000
    else:
        return float(value)
    
def find_close_matches_variable(this_jersey, dataset_nationality, threshold):
    close_matches = process.extract(this_jersey, dataset_nationality, limit=None)
    return [match[0] for match in close_matches if match[1] >= threshold]

def threshold_player_match(this_jersey, dataset_nationality):
    THRESHOLD_NUM = 89
    
    while THRESHOLD_NUM >= 50:
        matches = find_close_matches_variable(this_jersey, dataset_nationality, THRESHOLD_NUM)
        if matches:
            #print(f"Player is {this_jersey}. Found matches: {matches}. threshold is {THRESHOLD_NUM}")
            return matches, THRESHOLD_NUM
        else:
            THRESHOLD_NUM -= 1
    if THRESHOLD_NUM < 50:
        return [f"No matches found even with the lowest threshold."], this_jersey #jersey was {this_jersey}
    
    
def in_season_around(player_id, curr_season, age):
    if age == "Not Listed": age = 30
    age = int(age)
    year_before = int(curr_season) -1
    year_after = int(curr_season) + 1
    before_season = transfermarkt_data[(transfermarkt_data['PlayerID'] == player_id) & (transfermarkt_data['Season'] == year_before)]
    after_season =  transfermarkt_data[(transfermarkt_data['PlayerID'] == player_id) & (transfermarkt_data['Season'] == year_after)]
    if before_season.empty and after_season.empty:
        print("Both Empty")
        return -1, "Both Empty"
    if not before_season.empty and after_season.empty:
        print("Only Before Season")
        print(before_season.iloc[0]['Market Value'])
        if before_season.iloc[0]['Market Value'] != "-":
            if before_season.iloc[0]['Market Value'] == 0:
                return 0, "Only Before + Zero Before"
            else:
                if age >= 30:
                    return (float(before_season.iloc[0]['Market Value']) * .8), "Only Before"
                else:
                    return (float(before_season.iloc[0]['Market Value']) * 1.2), "Only Before"
        else:
            return 0, "Only Before X"
    if not after_season.empty and before_season.empty:
        print("Only After Season")
        print(after_season.iloc[0]['Market Value'])
        if after_season.iloc[0]['Market Value'] != "-":
            if after_season.iloc[0]['Market Value'] == 0:
                return 0, "Only After + Zero After"
            else:
                if age <= 30:
                    return (float(after_season.iloc[0]['Market Value']) * .8), "Only After"
                else:
                    return (float(after_season.iloc[0]['Market Value']) * 1.2), "Only After"

        else:
            return 0, "Only After X"
    if not before_season.empty and not after_season.empty:
        print("Need to average")
        #print("Before Season")
        print(before_season.iloc[0]['Market Value'])
        #print("After Season")
        print(after_season.iloc[0]['Market Value'])
        before_season_val = before_season.iloc[0]['Market Value']
        after_season_val = after_season.iloc[0]['Market Value']
        if before_season_val != "-" and after_season_val != "-":
            return ((float(before_season.iloc[0]['Market Value']) + float(after_season.iloc[0]['Market Value'])) / 2), "Used Average of Season Before and After"
        elif before_season_val != "-" and after_season_val == "-":
            if age >= 30:
                return (float(before_season.iloc[0]['Market Value']) * .8), "Used Season Before * .8 But Had Both"
            else:
                return (float(before_season.iloc[0]['Market Value']) * 1.2), "Used Season Before * 1.2 But Had Both"
        elif before_season_val == "-" and after_season_val != "-":
            if age <= 30:
                return (float(after_season.iloc[0]['Market Value']) * .8), "Used Season After * .8 But Had Both"
            else:
                return (float(after_season.iloc[0]['Market Value']) * 1.2), "Used Season After * 1.2 But Had Both"
        else:
            return 0, "Had Both but Both Were Empty"
        
def find_in_transfermarkt(player_name, season, country_code, transfermarkt_data):
    row = transfermarkt_data[(transfermarkt_data['Name'] == player_name) & (transfermarkt_data['Season'] == season) & (transfermarkt_data['Team 1 Code'] == country_code)]
    one_up_row = transfermarkt_data[(transfermarkt_data['Name'] == player_name) & (transfermarkt_data['Season'] == int(season)+1) & (transfermarkt_data['Team 1 Code'] == country_code)]
    one_down_row = transfermarkt_data[(transfermarkt_data['Name'] == player_name) & (transfermarkt_data['Season'] == int(season)-1) & (transfermarkt_data['Team 1 Code'] == country_code)]
    if not row.empty:
        player_id = row.iloc[0]['PlayerID']
        #team = row.iloc[0]['Team']
        age = row.iloc[0]['Age']
        #Here we should be checking if the row 
        marketval = row.iloc[0]['Market Value']
        if marketval != 0:
            print(f"Found in Curr Season: {marketval}")
            return marketval, "Found in Curr Season"
        else:
            new_mv, reason = in_season_around(player_id, season, age)
            print("Not in Curr Season")
            print(new_mv)
            print(reason)
            if new_mv == -1 or new_mv == 0 or new_mv == "-":
                print("Returning 0 and True")
                return 0, reason
            else:
                print("Returning 0 and False")
                return new_mv, reason
    elif not one_up_row.empty:
        player_id = one_up_row.iloc[0]['PlayerID']
        age = one_up_row.iloc[0]['Age']
        #marketval = one_up_row.iloc[0]['Market Value']
        new_mv, reason = in_season_around(player_id, season, age)
        if new_mv == -1 or new_mv == 0 or new_mv == "-":
            print("Returning 0 and True")
            return 0, reason
        else:
            print("Returning 0 and False")
            return new_mv, reason
    elif not one_down_row.empty:
        player_id = one_down_row.iloc[0]['PlayerID']
        age = one_down_row.iloc[0]['Age']
        new_mv, reason = in_season_around(player_id, season, age)
        if new_mv == -1 or new_mv == 0 or new_mv == "-":
            print("Returning 0 and True")
            return 0, reason
        else:
            print("Returning 0 and False")
            return new_mv, reason
    else:
        return 0, "Did Not Find in Curr Season, One Up, or One Down"

def go_thru_players(lookup_df, transfermarkt_data, start, stop):
    new_rows = []
    wrong_rows = []
    count = 0
    for index, row in lookup_df.iterrows():
        #print(count)
        #print(row)
        if count >= start and count < stop:
            name = row['Names_Found']
            original_jersey = row['ORIGINAL JERSEY']
            season = row['Season']
            country_code = row['Country Code']
            #return_case = row['Lookup Return Case']
            #other_name = row['Name(s) Found']
            #third_name = row['New Found Name']
            #Here we cannot use return_case = "Did Not Find"
            
            #if return_case == "Did Not Find":
                #print("Case 1")
                #If the name columns are both equal then we actually found the guy 
            #Unindenting because every row ought to be checked.
            if name == other_name and name == third_name:
                #print("Case 1.1")
                new_mv, valid_zero = find_in_transfermarkt(name, season, country_code, transfermarkt_data)
                if valid_zero != "Did Not Find":
                    new_row = row.to_dict()
                    new_row['Market Value'] = new_mv
                    new_row['Lookup Still Required?'] = "FALSE"
                    new_row['Lookup Return Case'] = valid_zero
                    new_row['Status'] = "SUCCESS"
                    new_rows.append(new_row)
                else:
                    new_row = row.to_dict()
                    new_row['Lookup Still Required?'] = "TRUE"
                    new_row['Lookup Return Case'] = "Failed Using Name"
                    new_row['Status'] = "FAIL"
                    wrong_rows.append(new_row)
            elif check_names_type(third_name) == "Single Name":
                new_mv, valid_zero = find_in_transfermarkt(third_name, season, country_code, transfermarkt_data)
                if valid_zero != "Did Not Find":
                    new_row = row.to_dict()
                    new_row['Market Value'] = new_mv
                    new_row['Lookup Still Required?'] = "FALSE"
                    new_row['Lookup Return Case'] = valid_zero
                    new_row['Status'] = "SUCCESS"
                    new_rows.append(new_row)
                else:
                    new_row = row.to_dict()
                    new_row['Lookup Still Required?'] = "TRUE"
                    new_row['Lookup Return Case'] = "Failed Using New Found Name"
                    new_row['Status'] = "FAIL"
                    wrong_rows.append(new_row)
            else:
                #dataset_nationality = transfermarkt_data[transfermarkt_data['Team 1 Code'] == country_code]['Name'].unique()
                #possible_names = threshold_player_match(original_jersey, dataset_nationality)
                
                #Hold off on trying to match these dudes 
                new_row = row.to_dict()
                new_row['Lookup Still Required?'] = "TRUE"
                new_row['Lookup Return Case'] = "Name Confusion"
                new_row['Status'] = "FAIL"
                wrong_rows.append(new_row)
            """
            else: 
                new_row = row.to_dict()
                new_rows.append(new_row)
            """
            #print(row['Match ID'])
        count +=1
    new_df = pd.DataFrame(new_rows)
    wrong_df = pd.DataFrame(wrong_rows)
    return new_df, wrong_df


In [20]:
lookup_df

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,Match ID,Match,Date,Competition,Nationality,Country Code,ORIGINAL JERSEY,Names_Found,Season,Status
0,0,0,0,1825,Cyprus vs Greece,24.09.22,Nations League 2022-2023,Cyprus,CY,Cham Kyriakou,Charalampos Kyriakou,2022,Success
1,1,1,1,1825,Cyprus vs Greece,24.09.22,Nations League 2022-2023,Cyprus,CY,Loizou,['Loizos Loizou'],2022,Success
2,2,2,2,1826,Greece vs Northern Ireland,27.09.22,Nations League 2022-2023,Northern Ireland,NI,Lavery,['Shayne Lavery'],2022,Success
3,3,3,3,1827,Kosovo vs Cyprus,27.09.22,Nations League 2022-2023,Kosovo,XK,Rrahmani,Amir Rrahmani,2022,Success
4,4,4,4,1827,Kosovo vs Cyprus,27.09.22,Nations League 2022-2023,Cyprus,CY,Cham Kyriakou,Charalampos Kyriakou,2022,Success
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5952,5952,5952,5952,2128,Mali vs Tunisia,25-03-2022,"World Cup Qualifiers, Africa, 2022",Mali,ML,C Traoré,Cheick Traoré,2022,Success
5953,5953,5953,5953,2139,Burundi vs Tanzania,04-09-2019,"World Cup Qualifiers, Africa, 2022",Burundi,BI,P Kwizera,Pierre Kwizera,2019,Success
5954,5954,5954,5954,2145,Cote d'Ivoire vs Mozambique,13-11-2021,"World Cup Qualifiers, Africa, 2022",Cote d'Ivoire,CI,H Traoré,['Hamed Junior Traorè'],2021,Success
5955,5955,5955,5955,3236,Russia vs Croatia,01-09-2021,"World Cup Qualifiers, Europe, 2022",Russia,RU,A Miranchuk,['Anton Miranchuk'],2021,Success


In [16]:

pd.set_option('display.float_format', '{:.2f}'.format)
lookup_df=pd.read_csv('Group 1 and 2 (for T) - Find their $ Info/updated_Group1_fixes_complete_ii.csv')
#lookup_df_wrong = lookup_df[lookup_df['Lookup Return Case'] == "Did Not Find"]
transfermarkt_data = pd.read_csv('MarketValues.csv')
transfermarkt_data['Market Value'] = transfermarkt_data['Market Value'].apply(convert_currency)
new_df, wrong_df = go_thru_players(lookup_df, transfermarkt_data, 0, 1)
#new_df = new_df.drop('Unnamed: 0.1.1', axis=1)
#new_df = new_df.drop('Unnamed: 0.1', axis = 1)
#wrong_df = wrong_df.drop('Unnamed: 0.1', axis=1)

KeyError: 'Lookup Return Case'